In [1]:
import pandas as pd
import random
import pickle
import math

In [3]:
random.seed(3214)

In [2]:
with open('../data/pickles/main_dict.pickle', 'rb') as f:
    main_dict = pickle.load(f)

In [4]:
df_train = pd.read_csv('../data/csv/train_2.csv', index_col=0)
df_test = pd.read_csv('../data/csv/test_2.csv', index_col=0)

In [8]:
def count_good_recommend(query_idx, answer_list, trhold):
    counter = 0
    query_id = int(df_test.iloc[query_idx].file[:-4])
    relevant = [query_id]
    query_tags = main_dict[query_id]['all_tags']
    for idx in answer_list:
        answer_id = int(df_train.iloc[idx].file[:-4])
        answer_tags = main_dict[answer_id]['all_tags']
        score = 0
        i = 0
        for tag in answer_tags:
            if tag in query_tags:
                i += 1
        if i == 0:
            continue
        
        precision = i / len(answer_tags)
        recall = i / len(query_tags)
        score = 2 * precision * recall / (precision + recall)
        if score >= trhold:        
            counter += 1
            relevant.append(answer_id)
    return counter, relevant

In [9]:
def count_ndcg(neighbors, relevant_list):
    dcg = 0
    idcg = 0
    for i in range(len(neighbors)):
        track = int(df_train.iloc[neighbors[i]].file[:-4])
        if track in relevant_list:
            dcg += 1 / math.log(i + 2, 2)
        idcg += 1 / math.log(i + 2, 2)
    ndcg = dcg / idcg
    return ndcg

In [35]:
mean_ap = 0
ndcg = 0
for i in range(100):
    random_recommendation = [random.randint(0, df_train.shape[0]) for j in range(10)]
    relevant_num, relevant_list = count_good_recommend(i, random_recommendation, trhold=0.4)
    p_at_10 = relevant_num / 10
    mean_ap += p_at_10
    ndcg += count_ndcg(random_recommendation, relevant_list[1:])
mean_ap /= 100 
mean_ndcg = ndcg / 100

In [36]:
mean_ap

0.005

In [37]:
mean_ndcg

0.005477892139975194

In [25]:
ap = 0
ndcg = 0
for i in range(100):
    mean_ap = 0
    ndcg = 0
    for i in range(100):
        test_track = df_test.iloc[i].file
        test_track_genre = main_dict[int(test_track[:-4])]['genre']
        genre_recommendations = list()
        while len(genre_recommendations) < 10:
            idx = random.randint(0, df_train.shape[0] - 1)
            track = df_train.iloc[idx].file
            track_genre = main_dict[int(track[:-4])]['genre']
            if track_genre == test_track_genre:
                genre_recommendations.append(idx)

        relevant_num, relevant_list = count_good_recommend(i, genre_recommendations, trhold=0.4)
    #     print(test_track, relevant_list)
        p_at_10 = relevant_num / 10
        mean_ap += p_at_10
        ndcg += count_ndcg(genre_recommendations, relevant_list[1:])
    mean_ap /= 100 
    mean_ndcg = ndcg / 100
    ap += mean_ap
    ndcg += mean_ndcg
print(ap/100, ndcg/100)

0.0547 0.07774172386379154


In [23]:
mean_ap

0.060999999999999985

In [24]:
mean_ndcg

0.07063325577848421

In [37]:
idx = random.randint(0, df_train.shape[0])

In [40]:
track = df_train.iloc[idx].track
    

In [71]:
main_dict[63804]['genre']

'Hip-Hop'

In [24]:
mean_ndcg

1.0

In [14]:
i = 2
[i for j in range(10)]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [45]:
df.shape

(7982, 338)